In [164]:
import boto3
import os,sys
import datetime as dt
import glob
import pandas as pd
pd.set_option('display.max_rows', 1000)
import hvplot.pandas
import holoviews as hv
import subprocess as sp

In [165]:
os.makedirs('/dev/shm/logs',exist_ok=True)
os.chdir('/dev/shm/logs')

In [166]:
!aws s3 sync s3://earthbigdata/logs/ /dev/shm/logs --quiet

In [167]:
def make_entry(ip_dict,date_dict,logfile):  
    with open(logfile,"r") as f:
        lines=f.readlines()
    date = lines[0].split('[')[1].split('+')[0].strip()
    date = dt.datetime.strptime(date,'%d/%b/%Y:%H:%M:%S')
    ipaddr=lines[0].split(']')[1].split('-')[0].strip()
    if not ipaddr in ip_dict:
        ip_dict[ipaddr]=[]
    if not date.date() in date_dict:
        date_dict[date.date()]=0
    ip_dict[ipaddr].append(date)
    date_dict[date.date()]+=1
    

In [168]:
ip_dict={}
date_dict={}

for log in glob.glob('ebd-covid19*'):
    make_entry(ip_dict,date_dict,log)

In [169]:
access=pd.DataFrame.from_dict(date_dict,orient='index')

In [170]:
access.sort_index(inplace=True,ascending=False)

In [171]:
access.hvplot.barh(padding=0.1,grid=True,ylabel='Access Times per day',xlabel='',title='EBD COVID 19 Access')

:Bars   [index]   (0)

In [150]:
ip_count={}
for i in ip_dict:
    ip_count[i]=len(ip_dict[i])

In [151]:
access_count=pd.DataFrame.from_dict(ip_count,orient='index')

access_count=access_count[access_count>=5].dropna()

access_count = access_count.sort_values(0,ascending=False)

access_count['nslookup']=''
for i in access_count.index:
    cmd='nslookup {}'.format(i)
    try:
        ret= sp.check_output(cmd.split())
        nslookup = ret.decode().split('name = ')[1].split('\n')[0].strip('.')
        # print(nslookup)
        access_count.loc[i,'nslookup']=nslookup
    except:
        pass

In [152]:
access_count.head(20)

,0,nslookup
24.218.227.219,444.0,c-24-218-227-219.hsd1.ma.comcast.net
52.44.93.197,116.0,ec2-52-44-93-197.compute-1.amazonaws.com
73.23.120.180,102.0,c-73-23-120-180.hsd1.fl.comcast.net
34.231.157.157,91.0,ec2-34-231-157-157.compute-1.amazonaws.com
34.232.127.140,78.0,ec2-34-232-127-140.compute-1.amazonaws.com
71.232.190.220,62.0,c-71-232-190-220.hsd1.ma.comcast.net
93.218.246.5,62.0,p5DDAF605.dip0.t-ipconnect.de
72.205.80.22,61.0,ip72-205-80-22.sb.sd.cox.net
73.143.234.232,52.0,c-73-143-234-232.hsd1.ma.comcast.net
70.191.91.13,51.0,ip70-191-91-13.sb.sd.cox.net


In [153]:
len(list(ip_dict.keys()))

1961

In [154]:
list(ip_dict.keys())[:10]

['62.216.208.62',
 '76.168.187.161',
 '76.238.199.154',
 '3.86.107.205 arn:aws:iam::811071659227:user/josefk 372A1AFA55544A43 REST.PUT.OBJECT index.html "PUT /ebd',
 '68.48.240.2',
 '77.88.5.49',
 '24.218.227.219',
 '68.56.148.231',
 '98.250.114.207',
 '76.80.178.3']

In [155]:
for i in list(ip_dict.keys()):
    if ip_dict[i][-1].date() == dt.datetime.now().date():
        if i in list(access_count.index):
            print(ip_dict[i][-1].date(),access_count.loc[i][0],'\t',access_count.loc[i].nslookup)

2020-03-31 5.0 	 aftr-62-216-208-62.dynamic.mnet-online.de
2020-03-31 11.0 	 cpe-174-103-120-159.columbus.res.rr.com
2020-03-31 5.0 	 23-127-164-61.lightspeed.sntcca.sbcglobal.net
